In [1]:

!ls -l /storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert

total 17496664
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com       709 May  2 12:27 config.json
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com       110 May  2 12:27 generation_config.json
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00001-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00002-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00003-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00004-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00005-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 436259736 May  2 12:25 pytorch_model-00006-of-00041.bin
-rw-rw-r-- 1 ext_jun_kato97_gmail_com ext_jun_kato97_gmail_com 4

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ext_ota_s_research_gmail_com/exp/miniconda3/envs/jupyter39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = "/storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False, torch_dtype="auto", device_map="auto")
tokenizer

LlamaTokenizer(name_or_path='/storage7/pretrain/Mixtral_3node_test/lr_1/epoch0-iter314999/convert', vocab_size=56320, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
model

Loading checkpoint shards: 100%|██████████| 41/41 [00:07<00:00,  5.16it/s]


MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(56320, 2048)
    (layers): ModuleList(
      (0-39): 40 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear(in_features=2048, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear(in_features=2048, out_features=4096, bias=False)
              (w2): Linear(in_features=4096, out_features=2048, bias=False)
              (w3): Linear(in_features=2048, out_features=4096, bias=False)
       

In [6]:
inputs = tokenizer("今日の天気は", return_tensors="pt").to(model.device)
inputs

{'input_ids': tensor([[    1,    32, 32132, 30152, 52207, 30154]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [7]:
model.generation_config.pad_token_id = model.generation_config.eos_token_id
outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.1)
outputs

tensor([[    1,    32, 32132, 30152, 52207, 30154, 38485,   292, 30643, 30170,
         31736, 30156, 30531, 30339, 31962,   292, 51090, 30152, 31028,   291,
         31736, 30156, 30531, 30221, 56215,   291, 32132, 30154, 30643, 30170,
         31736, 30153, 30259, 30159, 31123, 30274, 30393, 30152, 33215, 30156,
           291, 32844, 30237, 30170, 31736, 30156, 33701, 30189, 51246,   292,
         55481,   291, 49931, 54965, 30583, 30826]])

In [8]:
tokenizer.decode(outputs[0])

'<s>今日の天気は晴れ。朝から雨が降っています。昨日の夜、雨が降ったので、今日は朝から雨になると思っていたのですが、昼前から雨が降り出しました。そして、夕方には止んで'

In [9]:
inputs = tokenizer("日本で一番高い山は", return_tensors="pt", add_special_tokens=True).to(model.device)
print("inputs: ", inputs.input_ids)
outputs = model.generate(**inputs, max_new_tokens=50, repetition_penalty=1.1)
print("outputs: ", outputs[0])
tokenizer.decode(outputs[0], skip_special_tokens=False)

inputs:  tensor([[    1,    32, 30215, 30157, 32591, 31008, 30265, 30154]])
outputs:  tensor([    1,    32, 30215, 30157, 32591, 31008, 30265, 30154,   291, 32044,
        30265, 30546, 31423, 33215,   292, 32044, 30265, 30152, 33063, 30153,
        30163, 51590, 30281, 30152, 34488, 31227, 30513, 54965,   291, 32044,
        30265, 30155, 37084, 30162, 31259, 31067, 33874, 30156, 51240,   292,
        30193, 30203, 30166,   336, 32044, 30265, 51778, 30415,   337,   292,
        32044, 30265, 30152, 33063, 30153, 30163, 56215,   291])


'<s>日本で一番高い山は、富士山だそうです。富士山の麓にある山梨県の山中湖村には、富士山を眺める絶景スポットがあります。その名も「富士山展望台」。富士山の麓にあるので、'